In [1]:
%pwd

'c:\\Users\\aarav\\Downloads\\IRIS-MLops\\research'

In [2]:
import os
os.chdir('../')

In [22]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path 
    model_name: str 
    penalty: str             
    C: float 
    target_column: str  

In [4]:
# Configuration Manager

In [23]:
from IRIS_MLops.constants import *
from IRIS_MLops.utils.common import read_yaml, create_directories

In [24]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer 
          params = self.params.LogisticRegression 
          schema = self.schema.TARGET_COLUMN
          
          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
                root_dir=config.root_dir,
                train_data_path=config.train_data_path,
                test_data_path=config.test_data_path,
                model_name= config.model_name,
                penalty = params.penalty,
                C = params.C,
                target_column= schema.name
          )

          return model_trainer_config

In [7]:
# Components

In [25]:
import pandas as pd
import os 
from IRIS_MLops import logger 
from sklearn.linear_model import LogisticRegression
import joblib 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [26]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config 
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # Separate features and target variable
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]


        pipe = Pipeline(steps=[('scaler', StandardScaler())])

        # Combine preprocessing with the model
        model = Pipeline(steps=[('pipe', pipe),
                                ('model', LogisticRegression(penalty=self.config.penalty, C =self.config.C))])

        # Fit the model
        model.fit(train_x, train_y)

        # Save the trained model
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))


In [11]:
# Pipeline

In [29]:
try: 
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-13 15:41:45,117: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-13 15:41:45,120: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-13 15:41:45,123: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-13 15:41:45,124: INFO: common: created directory at: artifacts]


BoxKeyError: "'ConfigBox' object has no attribute 'target_column'"